#### **Linear Regression**
###### **In this tutorial we will deal with a regression problem and we will learn how to load the data, analyze the data and apply some pre-processing and apply a linear regression model(having only one layer). And further this tutorial we will extend this to a deep neural network. This will help us deep understanding of keras dense layer and activation functions.**

#### **Let's Start**

**Import Necessary Libraries**

In [ ]:
# To Ignore Warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt